In [26]:
import numpy as np
import pandas as pd
import seaborn as sns

In [28]:
tips = sns.load_dataset('tips')

In [29]:
tips.sample(5)

,total_bill,tip,sex,smoker,day,time,size
130,19.08,1.50,Male,No,Thur,Lunch,2
20,17.92,4.08,Male,No,Sat,Dinner,2
24,19.82,3.18,Male,No,Sat,Dinner,2
235,10.07,1.25,Male,No,Sat,Dinner,2
81,16.66,3.40,Male,No,Thur,Lunch,2


In [43]:
X = tips.iloc[:,1:]
y = tips.iloc[:,:1]
z = tips.iloc[:,1:2] # predict the tip also

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [47]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [49]:
X_train.head()

,tip,sex,smoker,day,time,size
228,2.72,Male,No,Sat,Dinner,2
208,2.03,Male,Yes,Sat,Dinner,2
96,4.00,Male,Yes,Fri,Dinner,2
167,4.50,Male,No,Sun,Dinner,4
84,2.03,Male,No,Thur,Lunch,2


In [52]:
trf = ColumnTransformer(
    transformers=[('ohe',OneHotEncoder(sparse= True ,drop='first'),['sex','smoker','day','time'])],
    remainder = 'passthrough'
)

In [53]:
X_train_trf = trf.fit_transform(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [57]:
X_test_trf = trf.transform(X_test)

In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [61]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
kn = KNeighborsRegressor()

In [63]:
lr.fit(X_train_trf,y_train)
dt.fit(X_train_trf,y_train)
kn.fit(X_train_trf,y_train)

KNeighborsRegressor()

In [69]:
lr_pred= lr.predict(X_test_trf)
dt_pred = dt.predict(X_test_trf)
kn_pred = kn.predict(X_test_trf)

In [70]:
from sklearn.metrics import r2_score

In [71]:
print("Linear regression : r2_score",r2_score(y_test,lr_pred))
print("Decision Tree regression : r2_score",r2_score(y_test,dt_pred))
print("KNN regression : r2_score",r2_score(y_test,kn_pred))




Linear regression : r2_score 0.6240808714290973
Decision Tree regression : r2_score 0.4639194377689291
KNN regression : r2_score 0.4965631529810982


# now let's apply bagging

In [76]:
from sklearn.ensemble import BaggingRegressor

bag = BaggingRegressor(estimator=None,n_estimators= 100,random_state =1)

In [84]:
bag.fit(X_train_trf,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(n_estimators=100, random_state=1)

In [85]:
y_pred = bag.predict(X_test_trf)

In [86]:
r2_score(y_test,y_pred)

0.5183750471287396

# Now Aoopliyng grid Seach CV

In [87]:
from sklearn.model_selection import GridSearchCV

In [94]:
parametes ={
    'estimator':[DecisionTreeRegressor(),LinearRegression(),KNeighborsRegressor()],
    'n_estimators':[40,60,80],
    'max_samples':[0.25,0.5],
    'max_features':[0.5,1.0],
    'bootstrap':[True,False],
    'bootstrap_features': [True, False]}


In [95]:
search = GridSearchCV(BaggingRegressor(random_state= 1, n_jobs= -1),param_grid=parametes,cv = 3, n_jobs= -1,verbose= 1)

In [96]:
search.fit(X_train_trf,y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


GridSearchCV(cv=3, estimator=BaggingRegressor(n_jobs=-1, random_state=1),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'estimator': [DecisionTreeRegressor(),
                                       LinearRegression(),
                                       KNeighborsRegressor()],
                         'max_features': [0.5, 1.0], 'max_samples': [0.25, 0.5],
                         'n_estimators': [40, 60, 80]},
             verbose=1)

In [99]:
print("Train r2_Score:",search.best_estimator_.score(X_train_trf,y_train))
print("Test r2_score:",search.best_estimator_.score(X_test_trf,y_test))

Train r2_Score: 0.5330901955828655
Test r2_score: 0.5570980616426717


In [100]:
search.best_estimator_

BaggingRegressor(bootstrap=False, bootstrap_features=True,
                 estimator=LinearRegression(), max_samples=0.5, n_estimators=80,
                 n_jobs=-1, random_state=1)

In [101]:
print("Best parametes:",search.best_estimator_)

Best parametes: BaggingRegressor(bootstrap=False, bootstrap_features=True,
                 estimator=LinearRegression(), max_samples=0.5, n_estimators=80,
                 n_jobs=-1, random_state=1)
